# Melatih Model untuk Memproses Dataset Gambar

Kita download terlebih dahulu dataset yang ingin kita gunakan

In [ ]:
from google.colab import files

upload = files.upload()

Saving Chessman-image-dataset.zip to Chessman-image-dataset.zip


Kemudia setelah terdownload kita bisa melakukan ekstrak terhadap dataset yang kita download

In [ ]:
import os
import zipfile as zp

local_file = '/tmp/Chessman-image-dataset.zip'
zip_ref = zp.ZipFile(local_file, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

Kita menggunakan fungsi listdir dari library os untuk melihat direktori dari file yang kita extract tadi

In [ ]:
os.listdir('/tmp/Chessman-image-dataset/Chess')

Kita perlu melihat berapa banyak gambar pada setiap direktori

In [ ]:
print('total Pion : ', len(os.listdir('/tmp/Chessman-image-dataset/Chess/Pawn')))
print('total Benteng : ', len(os.listdir('/tmp/Chessman-image-dataset/Chess/Bishop')))
print('total Kuda : ', len(os.listdir('/tmp/Chessman-image-dataset/Chess/Knight')))
print('total Ratu : ', len(os.listdir('/tmp/Chessman-image-dataset/Chess/Queen')))
print('total Raja : ', len(os.listdir('/tmp/Chessman-image-dataset/Chess/King')))

Untuk melihat contoh dari sebuah gambar dataset, bisa dengan menjalankan code dibawah

In [ ]:
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

img = image.load_img('/tmp/Chessman-image-dataset/Chess/King/00000145.png')
imgplot = plt.imshow(img)

Selanjutnya kita bisa mengimpor ImageDataGenerator dan aplikasikan augmentasi gambar pada dataset kita

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = os.path.join('/tmp/Chessman-image-dataset/Chess')
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.2,
    fill_mode='nearest',
    validation_split=0.1
)

dengan ImageDataGenerator kita tidak perlu susah payah membagi direktori secara manual, cukup dengan validation split

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150,150),
    batch_size=8,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150,150),
    batch_size=16,
    class_mode='categorical',
    subset='validation'
)

Kemudian kita membuat arsitektur model kita. Untuk model ini menggunakan 3 lapis convolution dan 2 hidden layer dengan 512 dan 256 buah unit perseptron

In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([
                                     # Note the input shape is the desired size of the image 150x150 with 3 bytes color
                                     tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
                                     tf.keras.layers.MaxPooling2D(2,2),
                                     tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
                                     tf.keras.layers.MaxPooling2D(2,2),
                                     tf.keras.layers.Dropout(0.4),
                                     tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
                                     tf.keras.layers.MaxPooling2D(2,2),
                                     tf.keras.layers.Dropout(0.4),
                                     # Flatten the results to feed into a DNN
                                     tf.keras.layers.Flatten(),
                                     tf.keras.layers.Dense(6, activation='softmax')
])

Setelah membuat arsitektur kita menentukan loss dan optimizer yang akan digunakan untuk model

In [ ]:
model.compile(
    optimizer=tf.optimizers.Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=50,
    verbose=2
)

Setelah model telah dilatih kita akan menampilkan grafik akurasi dan loss dari pelatihan model

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Akurasi Model')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss Model')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()